<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Time" data-toc-modified-id="Time-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Time</a></span></li><li><span><a href="#Quality" data-toc-modified-id="Quality-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Quality</a></span></li></ul></div>

In [1]:
## Benchmark pqlite

In [9]:
import pqlite
pqlite.__path__

import jina
from jina.math.distance import cdist

In [62]:
import random
import numpy as np
from pqlite import PQLite

#N = 100_000 # number of data points
Nt = 100_000
Nq = 1
D = 128 # dimentionality / number of features
top_k = 100
n_cells = 16

# 2,000 128-dim vectors for training
np.random.seed(123)
Xt = np.random.random((Nt, D)).astype(np.float32)  

# the column schema: (name:str, dtype:type, create_index: bool)
pq = PQLite(d_vector=D, n_cells=n_cells, n_subvectors=8, columns=[('x', float, True)])
pq.fit(Xt)

pq.add(Xt, ids=list(range(len(Xt))))

2021-11-11 18:34:55.219 | INFO     | pqlite.index:fit:90 - => start training VQ codec with 100000 data...
2021-11-11 18:34:58.738 | INFO     | pqlite.index:fit:93 - => start training PQ codec with 100000 data...
2021-11-11 18:35:54.326 | INFO     | pqlite.index:fit:96 - => pqlite is successfully trained!
2021-11-11 18:35:57.085 | DEBUG    | pqlite.storage.cell:_expand:148 - => total storage capacity is expanded by 100000 for 16 cells
2021-11-11 18:35:57.092 | DEBUG    | pqlite.storage.cell:insert:90 - => 100000 new items added


In [63]:
Nq = 1
np.random.seed(123)
query = np.random.random((Nq, D)).astype(np.float32)  # a 128-dim query vector

# without filtering
pq_dists, ids = pq.search(query, k=5)

### Time

In [56]:
%timeit  pq.search(query,  k=top_k)

22.9 ms ± 282 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [57]:
%timeit res = cdist(query, Xt, metric='euclidean')

13.7 ms ± 1.81 ms per loop (mean ± std. dev. of 7 runs, 100 loops each)


### Quality

In [58]:
true_distances = cdist(query, Xt, metric='euclidean').flatten()
true_ids = np.argsort(true_distances)[0:top_k]
print(true_distances)
print(true_ids)

[3.839339  4.1845713 4.407036  ... 4.4052653 4.3757777 4.1821375]
[89260 70691 93274 84864 60394 95720 46207 92802 35468 77584 56156  3003
 84646 41134 73512 32089 18211 97690 80260 10882 65422 94438 35128 67024
 40196  2506 93077 52740 42413 23847 61199 64831 47335 45481 97129 19182
 21738 29857 93055 35355 91492 83421 99556 19810 18318   313 94143 72034
 31437 55959  6201  2230 91150 60955 59653 23360 66408 54520 42348 11035
  7395 15708 42821 69674  5172 97853  9644 80891 60907 68004 36777  7780
 44418 64362  4165 92288 63462 61439 41443 24028 73553 91676 22931  7871
 36275 33260  9038 49749 72351 10396 48762  2270 74220 62055 36117 48051
 40144 68888  7269 68470]


In [64]:
pq_dists, pq_ids = pq.search(query,  k=top_k)
pq_ids = np.array([int(x) for x in pq_ids[0]])

In [65]:

def _precision(actual, desired, eval_at):
    if eval_at == 0:
        return 0.0
    actual_at_k = actual[:eval_at]
    ret = len(set(actual_at_k).intersection(set(desired)))
    sub = len(actual_at_k)

    return ret / sub

def _recall(actual, desired, eval_at):
    if eval_at == 0:
        return 0.0
    actual_at_k = actual[:eval_at]
    ret = len(set(actual_at_k).intersection(set(desired)))
    return ret / len(desired)


In [66]:
_precision(true_ids, pq_ids, top_k)

0.0